Project Overview: Build an ML system that predicts where courses from different universities are eligible for transfer credit using NLP and classification algorithms.

In [13]:
'''Core Environment Setup'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import warnings
import os
warnings.filterwarnings('ignore')

In [14]:
'''Data Collection Environment Setup'''
import requests
from bs4 import BeautifulSoup
import time
import re
import json
from urllib.parse import urljoin, urlparse

In [15]:
'''NLP Environment Setup'''
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
import nltk
import spacy

In [16]:
'''Deep Learning Environment Setup'''
from transformers import AutoTokenizer, AutoModel
import torch

In [17]:
'''Plotting'''
plt.style.use('seaborn-v0_8')
sb.set_palette("husl")

Project Configuration can now be completed.

In [18]:
'''Project Config'''
CONFIG = {
    'data_dir':'data/',
    'output_dir':'outputs/',
    'models_dir':'models/',
    'random_seed':42,
    'target_unis':['Purdue University', 'Cal', 'UMich', 'Georgia Tech', 'UIUC'],
    'target_departs':['CS', 'ECE', 'MATH', 'PHYS', 'ENGR'],
    'max_courses_per_dept':200,
    'min_descript_length':20
}

In [21]:
'''Project Directory Creation'''
for dir_name in [CONFIG['data_dir'], CONFIG['output_dir'], CONFIG['models_dir']]:
    os.makedirs(dir_name, exist_ok=True)